In [3]:
import torch

# 广播(broadcasting mechanism)

In [ ]:
a = torch.arange(3).reshape((3,1))
b = torch.arange(2).reshape((1,2))

a, b
a + b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [27]:
a = torch.arange(6).reshape((2,3))
b = torch.arange(3).reshape((3))
c = a + b

a, b, c

(tensor([[0, 1, 2],
         [3, 4, 5]]),
 tensor([0, 1, 2]),
 tensor([[0, 2, 4],
         [3, 5, 7]]))

# 指针

In [28]:
before = id(a)
a = a + b
id(a) == before

False

In [47]:
z = torch.zeros_like(a)
z[:] = a + b


In [48]:
before = id(a)
a += b
id(a) == before

True

# 数据预处理

In [52]:
import os
os.makedirs(os.path.join('..','data'),exist_ok=True)
data_file = os.path.join('..','data','house_tiny.csv')
data_file
with open(data_file,'w') as f:
    f.write('NumRooms,Alley,Price\n')
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('3,NA,140000\n')

In [ ]:
import pandas as pd

data = pd.read_csv(data_file)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,3.0,NaN,140000


为了处理缺失的数据，典型的方法包括差值和删除。

In [67]:
# 把NumRooms的缺失值替换为均值
data['NumRooms'] = data['NumRooms'].fillna(data['NumRooms'].mean())
data

,NumRooms,Alley,Price
0,3.0,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,3.0,NaN,140000


In [71]:
inputs = pd.get_dummies(data, dummy_na=True)
inputs

,NumRooms,Price,Alley_Pave,Alley_nan
0,3.0,127500,True,False
1,2.0,106000,False,True
2,4.0,178100,False,True
3,3.0,140000,False,True


现在inputs和outputs条目都是数值类型，可以将它转换为张量格式。

In [ ]:
inputs = torch.tensor(inputs.values)